Welcome to one of the collaborative Spark environments in ZHAW. You are not yet connected to Sparky by default. However, the necessary code template makes this a quick process. Keep in mind that you are sharing both the Jupyter environment and the Sparky cluster with others. Custom Python packages on the notebook/Spark driver side are installed with %pip install.

In [1]:
ticker = 'NESN.SW'
path = '/home/ubuntu/work/abd-fs23/fuxseb01/project/stock_data/'

In [2]:
import sparky

~~~ Sparky module loaded ~~~


In [3]:
%pip install yfinance

Error processing line 1 of /home/ubuntu/work/abd-fs23/fuxseb01/project/sparky-packages/lib/python3.11/site-packages/matplotlib-3.7.1-py3.11-nspkg.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 192, in addpackage
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 570, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of /home/ubuntu/work/abd-fs23/fuxseb01/project/sparky-packages/lib/python3.11/site-packages/matplotlib-3.7.1-py3.11-nspkg.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 192, in addpackage
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 570, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


In [4]:
PORT = 10089

In [5]:
# Helper functions

def get_stock_prices(ticker, counter, path):
    
    period = '5d'
    if counter != 0:
        period = '1d'
    
    df = yf.download(tickers=ticker, period=period, interval='1m')
    df['id'] = df.index.strftime('%Y-%m-%d %H:%M:%S')
    # Extract date from the index
    df['date'] = df.index.strftime('%Y-%m-%d')
    # Extract time from the index
    df['time'] = df.index.strftime('%H:%M:%S')
    
    #Create directory if required
    stock_directory = os.path.join(path, ticker)
    if not os.path.exists(stock_directory):
        os.makedirs(stock_directory)
    
    file_name = ticker +'_df.csv'
    path_to_file = os.path.join(path, file_name)
    #print('path to file')
    #print(path_to_file)
    df.to_csv(path_to_file, index=False, header=False)
    print(f"New DataFrame saved for ticker: {ticker}")
    return


def convert_data_to_json(data, ticker):
    for index, row in data.iterrows():
        identifier = index.strftime('%Y-%m-%d %H:%M:%S')
        # Extract date from the index
        date = index.strftime('%Y-%m-%d')
        # Extract time from the index
        time = index.strftime('%H:%M:%S')
        price = row['Adj Close']
        volume = row['Volume']

        d = {"id": identifier, "ticker": ticker, "date": date, "time": time, "price": price, "volume": volume}

        # Convert dictionary to json string
        s = json.dumps(d, ensure_ascii=False)

        # How can I now save s in a directory?
        #save_json_to_file(s, directory)

    return


def save_json_to_file(json_string, directory):
    tmp = json.loads(json_string)
    timestamp = tmp['id'].replace(':', '_')
    ticker = tmp['ticker']
    filename = timestamp + '_' + ticker + '.txt'
    stock_directory = os.path.join(directory, ticker)
    filepath = os.path.join(directory, ticker, filename)

    if not os.path.exists(stock_directory):
        os.makedirs(stock_directory)

    if not os.path.exists(filepath):
        with open(filepath, 'w') as file:
            file.write(json_string)
            print(f"JSON data saved to: {filepath}")
    else:
        print(f"JSON data with timestamp '{timestamp}' already exists. Skipping saving.")

In [6]:
import yfinance as yf
import socket
import time
from datetime import datetime
import os

def send(ticker, path):

  iter = 0

  while True:
    
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    client_socket.bind(('localhost', PORT))
    client_socket.listen(1000)
    print("Waiting for consumer to connect...")
    conn, addr = client_socket.accept()
    
    try:  
      while True:
        start = time.time()
                
        # ------> Get data from Yahoo Finance
        print('###########################')
        print('### Iteration : No. ' + str(iter) + ' ###')
        print('########################### \n')
        get_stock_prices(ticker, iter, path)
        
        fileName = ticker + '_df.csv'
        inputFile = os.path.join(path, fileName)
        
        ifile  = open(inputFile, 'r')
        
        for row in ifile.readlines() :
          #print("sending: " + row.rstrip() + "," + str(datetime.now()))
          message = row.rstrip() + ',' + str(datetime.now()) + "\n"
          message = message.encode()
          conn.send(message)
          # send data every 100msec
          time.sleep(0.01)
        
        iter += 1

    except Exception as e:
        print(str(e))
        conn.close()
        client_socket.close()
        continue
    finally:
        conn0.close()
        client_socket.close()

send(ticker, path)

Waiting for consumer to connect...
###########################
### Iteration : No. 0 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 2 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 3 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 4 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW


[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 43 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 44 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 45 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 46 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 47 ###
########################### 

[*********************100%****

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 85 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 86 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 87 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 88 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 89 ###
########################### 

[*********************100%****

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 127 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 128 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 129 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 130 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 131 ###
########################### 

[*********************100

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 169 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 170 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 171 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 172 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 173 ###
########################### 

[*********************100

...

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 925 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 926 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 927 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 928 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 929 ###
########################### 

[*********************100

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 967 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 968 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 969 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 970 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 971 ###
########################### 

[*********************100

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1009 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1010 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1011 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1012 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1013 ###
########################### 

[*******************

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1051 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1052 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1053 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1054 ###
########################### 

[*********************100%***********************]  1 of 1 completed
New DataFrame saved for ticker: NESN.SW
###########################
### Iteration : No. 1055 ###
########################### 

[*******************

NameError: name 'conn0' is not defined